In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import gensim
import datetime
import pandas as pd
from tqdm import tnrange
from tgraph import plot_loss
from IPython.display import clear_output
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

modelwv = Doc2Vec.load("./model/doc2vec_before_train.wv")
#embedding = from_pretrained(weights)

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence


torch.cuda.set_device(1)
print(torch.cuda.current_device())

## TODO

----

Filter data

In [ ]:
userLog = pd.read_csv("./clean_data/userlog_feature.csv").head(100000)
df['datetime'] = pd.to_datetime(df['year'].astype(str)+"-"+ df['month'].astype(str) +"-"+ df['day'].astype(str)+" "+ df['hour'].astype(str)+":00:00")
df.drop(collum)
isTrain = userLog['datetime']<datetime.datetime(2018,2,18)
test = userLog[~isTrain]
userLog = userLog[isTrain]
userLog1 = userLog.sort_values("datetime")


In [ ]:
userLog.shape

## Remove user that clicked too much

In [ ]:
userClickCount = userLog1.groupby("userCode")['project_id'].transform(lambda x: len(x))
userLog1['userClickCount'] = userClickCount
userLog2 = userLog1[userLog1['userClickCount']<500]
# (userLog2['userCode'].value_counts())

## Get only projects that has more than x user clicked

----

 Our assumption is that we more user click menas the project is better. (And are most likely will be clicked in the future)

In [ ]:
project_id_clicked = userLog2.groupby("project_id")['userCode'].transform(lambda x: len(x))
userLog2['project_clicked'] = project_id_clicked
# (userLog['project_id'].value_counts() <10).sum() to find number of project less than ten clicked
userLog3 = userLog2[userLog2['project_clicked']>300]

In [ ]:
#%%time
# What projects has this user been clicked.
userLogSeq = userLog3.groupby("userCode").agg({"project_id":lambda x: list(x)})
#userLogSeq

In [ ]:
#Filter out filter that we only click once in this database. (We'll use them later.)
userLogSeqFiltered = userLogSeq[userLogSeq['project_id'].apply(lambda x: len(x))>1]

## Prepare data format

### Map project id to project Index [0 to index]

In [ ]:
modelwv.wv.index2word.index('6709')

In [ ]:
userLogSeqFiltered2 = userLogSeqFiltered
userLogSeqFiltered2['project_id'] = userLogSeqFiltered['project_id'].apply(lambda x: [modelwv.wv.index2word.index(str(i)) for i in x])

In [ ]:
"""
Data format
[12, 14, 15, 17, 14, 17, 12], -> [12,14,15,16,14,17] : [14, 15, 17, 14, 17, 12]
"""
userLogSeqFiltered2['features'] = userLogSeqFiltered2['project_id'].apply(lambda x: x[:-1])
userLogSeqFiltered2['targets'] = userLogSeqFiltered2['project_id'].apply(lambda x: x[1:])



## Prepare into pad pack sequence


In [ ]:
# Pad and save length as function
userLogSeqFiltered3 = userLogSeqFiltered2
userLogSeqFiltered3['seq_len'] = userLogSeqFiltered3['features'].apply(len)
userSeq = userLogSeqFiltered3.sort_values('seq_len',ascending=False)[userLogSeqFiltered3['seq_len']<500] # Best should be 5k?
maxSeq = max(userSeq['seq_len'])

In [ ]:
# Pad with 0
userSeq['padded_features'] = userSeq.apply(lambda x: x['features']+[0]*(maxSeq - x['seq_len']),axis=1)
userSeq['padded_targets'] = userSeq.apply(lambda x: x['targets']+[0]*(maxSeq - x['seq_len']),axis=1)

In [ ]:
features = Variable(torch.Tensor(userSeq['padded_features']).long().cuda())
targets = Variable(torch.Tensor(userSeq['padded_targets'])).long().cuda()
seq_len = torch.Tensor(userSeq['seq_len'].values).long().cuda()

print(features.shape)
print(targets.shape)
print(seq_len.shape)

In [ ]:
userSeq['padded_features'].head()

## PadPack: (seq_len, ) #ต้องเรียงความยาวแล้ว

In [ ]:
weights = torch.FloatTensor(modelwv.wv.syn0)
weights.shape[0]

In [ ]:

hidden_size = 15

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        #self.projEmbedding = nn.Embedding(*weights.shape).from_pretrained(weight) # Spreading tuple into embedding dim
        self.projEmbedding = from_pretrained(weights, False)
        
        self.lstm = nn.LSTM(weights.shape[1],hidden_size, bidirectional = False)
        
        self.hidden2out = nn.Linear(hidden_size, weights.shape[0])#weights.shape[0])
            
        
        
        #self.init_hidden()
    
    def init_hidden(self, batch_size):
        #TODO: change to clicking context
        self.hidden = (Variable(torch.zeros(1, batch_size, hidden_size).cuda()),Variable(torch.zeros(1, batch_size, hidden_size).cuda()))
        
    def forward(self, inputs, lengths):

        batch_size = inputs.size()[0]
        
        embeds = self.projEmbedding(inputs)
        
        transposed_embeds = embeds.transpose(0,1)
        
        self.init_hidden(batch_size)
        
        packed_input = pack_padded_sequence(transposed_embeds, lengths.cpu().numpy())
        packed_outputs, (ht, ct) = self.lstm(packed_input, self.hidden)
        
        out, _ = pad_packed_sequence(packed_outputs)
        
        out = self.hidden2out(out)
        
        return out.transpose(0,1)#out.transpose(0,1)#
    
        

In [ ]:
def from_pretrained(embeddings, freeze=True):
    assert embeddings.dim() == 2, \
         'Embeddings parameter is expected to be 2-dimensional'
    rows, cols = embeddings.shape
    embedding = torch.nn.Embedding(num_embeddings=rows, embedding_dim=cols)
    embedding.weight = torch.nn.Parameter(embeddings)
    embedding.weight.requires_grad = not freeze
    return embedding

In [ ]:
model = LSTM().cuda()

In [ ]:
# Test neural networks
outputs = model(features[:50], seq_len[:50])
# values, indices = torch.max(outputs, 2)
# indices

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
# criterion = nn.MSELoss()
#TODO: binary cross entropy
def loss_function(outputs,targets):    
    bs, seq_len, _ = outputs.size()
    
    targets = targets[:,:seq_len]
    
    outputs_spread = outputs.contiguous().view(-1,weights.shape[0]).squeeze(1)
    targets_spread = targets.contiguous().view(-1,1).squeeze(1)
        
    return criterion(outputs_spread, targets_spread)

print(loss_function(outputs,targets[:50]))

# The project embedding were freezed
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

In [ ]:
outputs = model(features[:5], seq_len[:5])
loss = loss_function(outputs,targets[:5])
print(loss)
loss.backward()
optimizer.step()

In [ ]:
train_ratio = 0.9
trainIdx = int(features.shape[0]*train_ratio)
X_train, X_test = features[:trainIdx], features[trainIdx:]
y_train, y_test = targets[:trainIdx], targets[trainIdx:]
seq_train, seq_test = seq_len[:trainIdx], seq_len[trainIdx:]


## Training

In [ ]:

import bokeh
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook
output_notebook(bokeh.resources.INLINE)
from IPython.display import clear_output

def plot_loss(score_record):
    df = pd.DataFrame(score_record,columns=["train_loss","test_loss","MAP","batch","epoch","batch_size"])
    df['avg_loss'] = df.groupby("epoch")['train_loss'].transform("mean")
    df['avg_eval_loss'] = df.groupby("epoch")['test_loss'].transform("mean")
    df['avg_MAP'] = df.groupby("epoch")['MAP'].transform("mean")

    # create a new plot with a title and axis labels
    p = figure(title="Loss graph [epoch: %d]"%(df['epoch'].max()), x_axis_label='Time', y_axis_label='Loss')

    # add a line renderer with legend and line thickness
    p.line(df.index, df['train_loss'], legend="Train Loss.",  line_width=1, color="orange")

    p.line(df.index, df['avg_loss'], legend="Train Avg Loss.", line_width=4, color="red")
    
      # add a line renderer with legend and line thickness
    p.line(df.index, df['test_loss'], legend="Test Loss.",  line_width=1 )

    p.line(df.index, df['avg_eval_loss'], legend="Test Avg Loss.", line_width=4, color="green")
    
    p.line(df.index, df['MAP'], legend="MAP@7",  line_width=1, color="black")
    
    p.legend.click_policy="hide"
    # show the results
    show(p)

In [ ]:
epoch = 0
score_record = []

In [ ]:
# Training 
import math

batch_size = 64
batch_per_epoch = math.ceil(X_train.shape[0]/batch_size)
epoch_num = 100


for epoch in tnrange(epoch,epoch_num):
    for batch_iter in tnrange(batch_per_epoch):
        start, end = batch_iter*batch_size, (batch_iter+1)* batch_size
        X_training_batch, y_training_batch, seq_training_batch  = X_train[start:end], y_train[start:end], seq_train[start:end]
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(X_training_batch, seq_training_batch)
        loss = loss_function(outputs, y_training_batch)
        
        eval_outputs = model(X_test, seq_test)
        eval_loss = loss_function(eval_outputs, y_test)
        
        MAP_score, _  = calMap(model)
        print(MAP_score)
        #MAP_score = 0
        score_record.append((loss.data[0],eval_loss.data[0],MAP_score,batch_iter,epoch,batch_size))    
        
        loss.backward()
        optimizer.step()
    clear_output()
    plot_loss(score_record)

In [ ]:
# TODO extract most probably correct projects from deep
def score_AP(correct, recommended, n=7):
    recommended = list(recommended)[:7]
    assert(len(set(recommended))==len(recommended)) # Recommended should be unique    
    precisions = []
    correct_score = 1
    for idx, recommend_item in enumerate(recommended):
        if str(recommend_item) not in correct: precisions.append(0)
        else:                         
            precisions.append(correct_score/(idx+1))
            correct_score+=1
    return sum(precisions)/min(len(correct),n)

In [ ]:
import pandas as pd
test_userLog = pd.read_csv("./clean_data/deeplstmfeatures.csv")
test_padded_features =  test_userLog['padded_features'].apply(lambda x: eval(x))
test_userLog['ans_project_id'] = test_userLog['ans_project_id'].apply(lambda x: eval(x))
test_userLog['ans_project_id'] = test_userLog['ans_project_id'].apply(lambda x: [modelwv.wv.index2word.index(str(i)) for i in x if str(i) in modelwv.wv.index2word])
test_userLog = test_userLog[test_userLog['ans_project_id'].apply(lambda x: len(x)>0)]
test_seq_len = test_userLog['seq_len'].values

#filtered_submission_users = userLog[ans_project_id.apply(len) > 0].set_index("userCode")


test_features_tensor = Variable(torch.Tensor(test_padded_features).long()).cuda()
test_seq_len_tensor = torch.Tensor(test_seq_len).long().cuda()

In [ ]:
import numpy as np
def calMap(model, n = 10):
    outputs = model(test_features_tensor[:n], test_seq_len_tensor[:n]).cpu()
    preds = np.array([o[seq-1].data.numpy() for o, seq in zip(outputs,test_seq_len[:n])])
    idxes = [(-preds[1]).argsort()[:30] for pred in preds]
    testingUser = test_userLog.head(n)
    testingUser['preds'] = idxes
    #print(testingUser['preds'])
    testingUser.apply(lambda row: [pred for pred in row['preds'] if pred not in row['ans_project_id']],axis=1)
    testingUser['MAP'] = testingUser.apply(lambda x: score_AP(x['ans_project_id'],x['preds']),axis=1)
    return testingUser['MAP'].mean(),testingUser
score, df = calMap(model)

In [ ]:
df.head()

In [ ]:
torch.save(model.state_dict(), "./model/LSTMfuture_v4.weight")